In [1]:
from model import Model
import tensorflow as tf

In [2]:
model = Model(
    memorySize = 60,
    windowSize = 60,
    threshold = 100,
    inputDimension = 1,
    hiddenStateSize = 10,
    extremeValueIndex = 2.1,
    optimizer = tf.keras.optimizers.Adam(),
    extremeLossWeight = 1.0
)